In [1]:
import snowflake.connector
import snowflake.connector as sc
import os
from cryptography.hazmat.primitives import serialization

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

True

#  snowflake_helper

In [11]:
# --- snowflake_helper.py (updated) ---
import os
import threading
from queue import Queue
from contextlib import contextmanager
import snowflake.connector
import pandas as pd

class SnowflakeHelper:
    def __init__(self, database, schema, minconn=1, maxconn=5):
        self.database = database
        self.schema = schema
        self.minconn = minconn
        self.maxconn = maxconn
        self.pool = Queue(maxconn)
        self.lock = threading.Lock()
        self._initialize_pool()

    def _initialize_pool(self):
        """Initialize the connection pool with minconn connections."""
        for _ in range(self.minconn):
            self.pool.put(self._create_connection())

    def _create_connection(self):
        """Create a new Snowflake connection using env variables (key-pair or pwd; both supported)."""
        conn = snowflake.connector.connect(
            user=os.getenv("SNOWFLAKE_USER"),
            account=os.getenv("SNOWFLAKE_ACCOUNT"),
            # warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
            role=os.getenv("SNOWFLAKE_ROLE"),
            database=self.database,
            schema=self.schema,
            # If using key-pair/JWT:
            authenticator=os.getenv("SNOWFLAKE_AUTHENTICATOR"),
            private_key_file=os.getenv("SNOWFLAKE_PRIVATE_KEY"),
        )

        print(f'Selected warehouse: {os.getenv("SNOWFLAKE_WAREHOUSE")}')

        # # ✅ Explicitly activate the warehouse for the current session
        # warehouse = os.getenv("SNOWFLAKE_WAREHOUSE")
        # if warehouse:
        #     # Clean warehouse name to avoid syntax errors
        #     warehouse = warehouse.strip().strip('"').strip("'").strip(",")
        #     try:
        #         cur = conn.cursor()
        #         cur.execute(f'USE WAREHOUSE "{warehouse}";')
        #         cur.close()
        #         print(f"✅ Active warehouse set: {warehouse}")
        #     except Exception as e:
        #         print(f"⚠️ Warning: Failed to activate warehouse {warehouse}: {e}")

        return conn


    @contextmanager
    def _get_connection(self):
        """Fetch a connection from the pool and return it after use."""
        conn = None
        try:
            conn = self.pool.get(block=True, timeout=10)
            yield conn
        finally:
            # If connection object exists, return it to the pool.
            # (If it's broken, the next execute will error and trigger retry replacement.)
            if conn is not None:
                try:
                    self.pool.put(conn, block=False)
                except Exception:
                    try:
                        conn.close()
                    except Exception:
                        pass

    def _replace_connection(self):
        """Replace a potentially bad connection with a fresh one (best-effort)."""
        try:
            self.pool.put(self._create_connection(), block=False)
        except Exception:
            pass

    def execute_query(self, query: str, params=None, as_df: bool = False, max_retries: int = 1):
        """
        Execute a SQL query using the pool and return (result, error_message).

        Returns:
            tuple[result, str|None]
              - If as_df=True: (pd.DataFrame, error)
              - Else: (list[tuple], error)
        """
        # Do NOT strip semicolons; Snowflake accepts them.
        params = params or {}
        last_err = None

        for attempt in range(max_retries + 1):
            cursor = None
            try:
                with self._get_connection() as conn:
                    cursor = conn.cursor()
                    cursor.execute(query, params)

                    if as_df:
                        df = cursor.fetch_pandas_all()
                        return df, None
                    else:
                        rows = cursor.fetchall()
                        return rows, None

            except Exception as e:
                last_err = str(e)
                # Best-effort: try to replace a potentially broken connection for next attempt
                if attempt < max_retries:
                    self._replace_connection()
                else:
                    # Final failure → return typed empty result + error
                    if as_df:
                        return pd.DataFrame(), last_err
                    else:
                        return [], last_err
            finally:
                if cursor is not None:
                    try:
                        cursor.close()
                    except Exception:
                        pass

# Example - database/schema passed at initialization
sf = SnowflakeHelper(database="ahp", schema="qa", minconn=2, maxconn=10)

Selected warehouse: DEVELOPMENT_QA_WH
Selected warehouse: DEVELOPMENT_QA_WH


In [12]:
conn = snowflake.connector.connect(
        user=os.getenv("SNOWFLAKE_USER"),
        account=os.getenv("SNOWFLAKE_ACCOUNT"),
        # warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
        role=os.getenv("SNOWFLAKE_ROLE"),
        database='ahp',
        schema='qa',
        # If using key-pair/JWT:
        authenticator=os.getenv("SNOWFLAKE_AUTHENTICATOR"),
        private_key_file=os.getenv("SNOWFLAKE_PRIVATE_KEY"),
    )


In [18]:
import pandas as pd

cur = conn.cursor()
cur.execute("SHOW ROLES;")
roles_data = cur.fetchall()
columns = [col[0] for col in cur.description]
roles_df = pd.DataFrame(roles_data, columns=columns)
cur.close()

print("🎭 Available Roles:")
print(roles_df[["name", "owner", "is_current"]])


🎭 Available Roles:
                             name         owner is_current
0                    ACCOUNTADMIN                        N
1                         ANALYST  ACCOUNTADMIN          N
2              DB_AHP_TRANSFORM_R  ACCOUNTADMIN          N
3             DB_AHP_TRANSFORM_RW  ACCOUNTADMIN          N
4           DB_ANALYST_AHP_PROD_R  ACCOUNTADMIN          N
5             DB_ANALYST_AHP_QA_R  ACCOUNTADMIN          N
6   DB_ANALYST_SUPPLYCOPIA_PROD_R  ACCOUNTADMIN          N
7     DB_ANALYST_SUPPLYCOPIA_QA_R  ACCOUNTADMIN          N
8               DB_DEV_AHP_DEV_RW  ACCOUNTADMIN          N
9               DB_DEV_AHP_PROD_R  ACCOUNTADMIN          N
10                DB_DEV_AHP_QA_R  ACCOUNTADMIN          N
11      DB_DEV_SUPPLYCOPIA_DEV_RW  ACCOUNTADMIN          N
12      DB_DEV_SUPPLYCOPIA_PROD_R  ACCOUNTADMIN          N
13        DB_DEV_SUPPLYCOPIA_QA_R  ACCOUNTADMIN          N
14               DB_MS_AHP_PROD_R  ACCOUNTADMIN          N
15              DB_MS_AHP_PROD_RW  AC

In [19]:
cur = conn.cursor()
cur.execute("SHOW WAREHOUSES;")
warehouses_data = cur.fetchall()
columns = [col[0] for col in cur.description]
warehouses_df = pd.DataFrame(warehouses_data, columns=columns)
cur.close()

print("🏢 Available Warehouses:")
print(warehouses_df[["name", "state", "size", "auto_suspend", "auto_resume", "created_on"]])


🏢 Available Warehouses:
                           name      state     size  auto_suspend auto_resume  \
0             DATAMANAGEMENT_WH  SUSPENDED  X-Small           300        true   
1  SYSTEM$STREAMLIT_NOTEBOOK_WH  SUSPENDED  X-Small            60        true   

                        created_on  
0 2025-09-30 01:26:17.701000-07:00  
1 2025-07-07 03:16:31.311000-07:00  


In [ ]:
cur = conn.cursor()
cur.execute("SHOW GRANTS TO ROLE ML_LLM;")
rows = cur.fetchall()
for r in rows:
    print(r)
cur.close()

(datetime.datetime(2025, 9, 29, 20, 37, 25, 802000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), 'USAGE', 'ROLE', 'DB_MS_AHP_PROD_R', 'ROLE', 'ML_LLM', 'false', 'ACCOUNTADMIN')
(datetime.datetime(2025, 9, 29, 20, 37, 25, 284000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), 'USAGE', 'ROLE', 'DB_MS_AHP_QA_R', 'ROLE', 'ML_LLM', 'false', 'ACCOUNTADMIN')
(datetime.datetime(2025, 9, 29, 20, 37, 26, 795000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), 'USAGE', 'ROLE', 'DB_MS_SUPPLYCOPIA_PROD_R', 'ROLE', 'ML_LLM', 'false', 'ACCOUNTADMIN')
(datetime.datetime(2025, 9, 29, 20, 37, 26, 264000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), 'USAGE', 'ROLE', 'DB_MS_SUPPLYCOPIA_QA_R', 'ROLE', 'ML_LLM', 'false', 'ACCOUNTADMIN')


True

In [22]:
warehouse_name = "DATAMANAGEMENT_WH"  # change if needed

cur = conn.cursor()
try:
    cur.execute(f'USE WAREHOUSE "{warehouse_name}";')
    print(f"✅ Successfully set active warehouse: {warehouse_name}")
except Exception as e:
    print(f"❌ Failed to set warehouse {warehouse_name}: {e}")
finally:
    cur.close()


✅ Successfully set active warehouse: DATAMANAGEMENT_WH


In [17]:
cur.fetch_pandas_all()

NotSupportedError: Unknown error

In [16]:
roles_df

NameError: name 'roles_df' is not defined

In [ ]:
# # Example - database/schema passed at initialization
# sf = SnowflakeHelper(database="ahp", schema="qa", minconn=2, maxconn=10)

In [ ]:
# cqo_chatbot_case_level
# cqo_chatbot_product_level
# cqo_chatbot_robotics

In [23]:
tbl_name = "cqo_chatbot_case_level"
df, error = sf.execute_query("SELECT * from cqo_chatbot_case_level limit 10", as_df=False)

Selected warehouse: DEVELOPMENT_QA_WH


In [29]:
import pandas as pd

cur = conn.cursor()
try:
    # ✅ Make sure you have an active warehouse
    cur.execute('USE WAREHOUSE "DEVELOPMENT_QA_WH";')

    # ✅ Run your query
    cur.execute("SELECT * FROM cqo_chatbot_case_level LIMIT 10;")

    # ✅ Fetch results
    rows = cur.fetchall()

    # ✅ Get column names
    columns = [col[0] for col in cur.description]

    # ✅ Convert to DataFrame (optional)
    df = pd.DataFrame(rows, columns=columns)

    print("✅ Query executed successfully!")
    print(df.head())

except Exception as e:
    print(f"❌ Error executing query: {e}")

finally:
    cur.close()


✅ Query executed successfully!
                                         EMR_P_EVENT EMR_HOSPITAL_ROLLUP  \
0         MARYVIEW MEDICAL CENTER-920242285156-28750       HAMPTON ROADS   
1  MERCY HEALTH - ST. JOSEPH WARREN HOSPITAL-7212...          YOUNGSTOWN   
2  MERCY HEALTH - ST. ELIZABETH BOARDMAN HOSPITAL...          YOUNGSTOWN   
3  MERCY HEALTH - FAIRFIELD HOSPITAL-616233478346...          CINCINNATI   
4              LOURDES HOSPITAL-101252021123-0FT44ZZ            KENTUCKY   

                 EMR_FACILITY_NAME  \
0          MARYVIEW MEDICAL CENTER   
1       ST. JOSEPH WARREN HOSPITAL   
2  ST. ELIZABETH BOARDMAN HOSPITAL   
3               FAIRFIELD HOSPITAL   
4                 LOURDES HOSPITAL   

                                          EMR_MS_DRG  \
0                                            UNKNOWN   
1                                            UNKNOWN   
2  788-CESAREAN SECTION WITHOUT STERILIZATION WIT...   
3                                            UNKNOWN   
4  

In [ ]:
error

In [ ]:
error

In [ ]:
error

In [9]:
# --- List Active Warehouses using SnowflakeHelper ---
query = "SHOW WAREHOUSES;"
values, err = sf.execute_query(query, as_df=False)
values


Selected warehouse: DEVELOPMENT_QA_WH


[]

In [10]:
values 

[]

In [5]:
values 

[]

In [ ]:
df

In [ ]:
df 

In [ ]:
df

In [ ]:
query = "SHOW WAREHOUSES;"
# query = "ALTER WAREHOUSE DATAMANAGEMENT_WH RESUME;"
values, err = sf.execute_query(query, as_df=False)
values

In [ ]:
values

In [ ]:
df = pd.DataFrame(values)

In [ ]:
df 

In [ ]:
df 

In [ ]:
df

In [ ]:
error

In [ ]:
df 

In [ ]:
df = sf.execute_query(query, as_df=True)
column_list = df['COLUMN_NAME'].unique().tolist()